# Cryptography

## Secure Multiparty Computation - part 1

# Intro

**A Pragmatic Introduction to Secure Multi-Party Computation** by David Evans, Vladimir Kolesnikov, Mike Rosulek

To see the notebook as a slide-show, use [RISE](https://rise.readthedocs.io/en/stable/)

## Agenda

### Motivation

* Auctions
* Procurement
* Benchmarking
* Data Mining
* Voting
* Any computable function/functionality

### Tools

* (verifiable) secret sharing
* oblivious transfer
* (homomorphic) commitments
* zero-knowledge proofs 


![oblivious transfer](img/ot.png)

In [10]:
import math
import random
from sympy import randprime, isprime, Mod

In [12]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [13]:
from sympy import primefactors

**The factoring experiment** $\textsf{Factor}_{\mathcal{A}, GenModulus}(n)$
1. $(N, p, q) := GenModulus(1^n)$

2. $(p', q') = \mathcal{A}(N)$

3. Output: 1 if $p' \cdot q' = N$ and $p', q' > 1$

**Definition 7.45** We say that factoring is hard relative to **GenModulus** if for all probabilistic polynomial-time algorithms $\mathsf{A}$ there exists a negligible function **negl** such that 
$$P[\textsf{Factor}_{\mathcal{A}, GenModulus}(n) = 1] \leq negl(n)$$

**The factoring experiment** $\textsf{Factor}_{\mathcal{A}, GenModulus}(n)$
1. $(N, p, q) := GenModulus(1^n)$

2. $(p', q') = \mathcal{A}(N)$

3. Output: 1 if $p' \cdot q' = N$ and $p', q' > 1$

**Definition 7.45** We say that factoring is hard relative to **GenModulus** if for all probabilistic polynomial-time algorithms $\mathsf{A}$ there exists a negligible function **negl** such that 
$$P[\textsf{Factor}_{\mathcal{A}, GenModulus}(n) = 1] \leq negl(n)$$

In [14]:
def GenModulus(w):
    n = len(w)
    p = randprime(2 ** n, 2 ** (n+1))
    q = randprime(2 ** n, 2 ** (n+1))
    N = p * q
    return N, p, q

In [15]:
def Factor(n):
    N, p, q = GenModulus("1" * n)
    pp, qq = A(N)
    if pp > 1 and qq > 1 and pp * qq == N:
        return 1
    else:
        return 0

In [16]:
def A(N):
    factors = primefactors(N)
    return factors[0], factors[1]

In [ ]:
%%time
sec_param = 52
num_of_succ = 0
tries = 100

for i in range(tries):
    num_of_succ = num_of_succ + Factor(sec_param // 2)

num_of_succ/tries

In [17]:
def GenRSA(w):
    n = len(w)
    N, p, q = GenModulus(w)
    m = (p-1) * (q-1)
    e = 2 ** 16 + 1
    d = modinv(e, m)
    return N, e, d, p, q
# N, e - public key
# N, d - private key

def enc(x, N, e):
    return x ** e % N

def dec(c, N, d):
    return c ** d % N

N, e, d, p, q = GenRSA("111111111")
#print("private key: ", d)
#print("public key: ", N, e)

xx  = 120 * p % N #random.randint(2, N)

y = enc(xx, N, e)
x = dec(y, N, d)
print(N, e, y)

512893 65537 467343


**The RSA experiment** $\textsf{RSA-inv}_{\mathcal{A}, GenRSA}(n)$
1. $(N, e, d) := GenRSA(1^n)$

2. choose $y \leftarrow Z_N^*$

2. $x = \mathcal{A}(N, e, y)$

3. Output: 1 if $x^e = y \bmod N$

**Definition 7.46** We say that RSA problem is hard relative to **GenRSA** if for all probabilistic polynomial-time algorithms $\mathsf{A}$ there exists a negligible function **negl** such that 
$$P[\textsf{RSA-inv}_{\mathcal{A}, GenRSA}(n) = 1] \leq negl(n)$$

In [18]:
def randomZnElement(N):
    g = N
    while math.gcd(g, N) != 1:
        g = random.randint(2, N)
    return g

# Naive Identification Scheme

## Password Scheme

Peggy $(x, f(x))$, Vic $f(x)$
1. Peggy:
    * sends $x$ to Vic
2. Vic:
    * checks if $f(x)$ is correct
  

## Identification based on Public-Key Encryption

Peggys $(x, f(x))$, Vic: $f(x)$

1. Vic:
    * selects $m$
    * encrypts $c = Enc(f(x), m)$
    * sends $c$ to Peggy
2. Peggy:
    * decrypts $m' = Dec(x, c)$
    * sends $m'$ to Vic
3. Vic:
    * accepts the identity of Peggy iff $m=m'$
    
Dishonest Vic :
1. captures an encryption $c$ that was sent to Peggy and then Vic can sent $c$ as a challenge
2. Peggy decrypts $m' = Dec(x, c)$
3. Vic learns the value of $m' = m$

Before the protocol:
- f(x)
- c
After the protocol:
- he knows that $m = Dec(x, c)$

![Naive authentication scheme](img/auth-enc.png)

A (Peggy) has:
    - secret key $x$
    - public key $f(x)$, for one-way function $f$
    
B (Vic) has:
    - public key $f(x)$
    
    
A's input: x
$$F_A = 0$$


B's input: $f(x) = y$
$$F_B(x, y) = (y = f(x)) ? 1:0$$


**Completenes**: If Peggy knows the prover's secret, then Vic will always accept Peggy's proof.

**Soundness**: If Peggy can convince Vic with reasonable probability, then she know the prover's secret.

If Peggy does not know the secret then with high probability Vic will detect that

**Zero-knowledge** (intuition): whatever the verifier can efficiently compute after interacting with the prover, can be efficiently simulated without interaction.

Performing the protocol with the Prover (Peggy) should not give any new information to Verifier (Vic)